In [14]:
import json

def create_json_from_dialogue(source_file, output_file, ignore_prefixes):
    # Initialize an empty list to store the entries
    entries = []

    # Open the source file and read all lines
    with open(source_file, 'r') as file:
        lines = file.readlines()

    # Initialize variables to store the input and output text
    input_text = ""
    output_text = ""
    current_speaker = None

    for line in lines:
        line = line.strip()  # Remove any leading/trailing whitespace

        # Check if the line starts with any prefix in ignore_prefixes
        if any(line.startswith(prefix) for prefix in ignore_prefixes):
            continue  # Skip this line if it matches any of the ignore prefixes

        # Check if the line starts with 'S:' or 'T:'
        if line.startswith("Boso."):
            # If current_speaker is 'T', save the previous conversation entry
            if current_speaker == "T" and input_text and output_text:
                entry = {
                    "instruction": "You are Anselm, in dialogue with a student about an aspect of your philosophy",
                    "input": input_text.strip(),
                    "output": output_text.strip()
                }
                entries.append(entry)
                input_text, output_text = "", ""  # Reset for the next entry

            # Start capturing input for the new dialogue
            input_text = line[6:].strip()
            current_speaker = "S"
        elif line.startswith("Anselm."):
            # If current_speaker is 'S', start capturing output
            if current_speaker == "S" and input_text:
                current_speaker = "T"
                output_text = line[8:].strip()
            else:
                # Continue capturing output from 'T'
                output_text += " " + line[8:].strip()
        else:
            # Continue capturing lines for the current speaker
            if current_speaker == "S":
                input_text += " " + line
            elif current_speaker == "T":
                output_text += " " + line

    # Add the last entry if available
    if input_text and output_text:
        entry = {
            "instruction": "You are Anselm, in dialogue with a student about an aspect of your philosophy",
            "input": input_text.strip(),
            "output": output_text.strip()
        }
        entries.append(entry)

    # Write the entries to the output JSON file
    with open(output_file, 'w') as json_file:
        json.dump(entries, json_file, indent=4)

    print(f"JSON file '{output_file}' created successfully.")


# Example usage
source_file = 'anselmcur.txt'  # Replace with your source file name
output_file = 'anselmcur.json'  # Replace with your desired output file name
ignore_prefixes = ["1", "Freedom of Choice", "CHAPTER", "2","3","4","5","6","7","8","9"]  # Add any prefixes to ignore

create_json_from_dialogue(source_file, output_file, ignore_prefixes)


JSON file 'anselmcur.json' created successfully.


In [ ]:
dataset = my_dataset
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass


dataset = dataset.map(formatting_prompts_func, batched = True,)